In [38]:
import xml.etree.ElementTree as ET
import matplotlib.pyplot as plt
import glob
import pickle
import numpy as np
import sys, os
import pylab
import time 
import scipy.misc
from skimage.transform import resize
import matplotlib
import re
from PIL import Image

import keras
from keras.models import Sequential
from keras.models import Model
from keras.layers import Dense, Activation, Flatten, Lambda,Input, concatenate, Conv2DTranspose
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization 
from keras.utils import np_utils 
from keras import optimizers 
from keras.models import load_model
from keras.models import model_from_json

import keras.backend as K 
print('Image ordering is tf check: ', K.common.image_dim_ordering())

Image ordering is tf check:  tf


In [22]:
# Load Model
json_string = pickle.load( open( "model_json.pkl", "rb" ) )
model = model_from_json(json_string)
model.load_weights('model_weights')

In [41]:
# Load test images
test_image_1 = Image.open("test-image-1.png")#.convert("LA")
# Convert test image to array
test_image_1 = np.array(test_image_1)
test_image_1 = np.expand_dims(test_image_1, axis=0)
#plt.imshow(test_image_1)

In [42]:
#predicting for the image with the trained model
yout = model.predict(test_image_1)

#Plotting input image
plt.imshow(test_image_1.reshape(512,512),cmap = 'gray')
plt.title('Input X')
plt.show()

#Plotting input image with annotated nodule
plt.imshow(test_image_1.reshape(512,512),cmap = 'gray')
plt.title('Input X with annotation')

#bounding box for annotation 
annot = np.where(Y_test[choice:choice+1,:,:,0]==1)
startx = annot[2][0]*32
endx = startx + 32
starty = annot[1][0]*32
endy = starty + 32

lines_x = []
lines_y = []

toph_x = [xx for xx in range(startx,endx+1)]
toph_y = [starty for new in toph_x]
lines_x.append(toph_x)
lines_y.append(toph_y)

bottomh_x = [xx for xx in range(startx,endx+1)]
bottomh_y = [endy for new in bottomh_x]
lines_x.append(bottomh_x)
lines_y.append(bottomh_y)

leftv_y = [yy for yy in range(starty,endy+1)]
leftv_x=[startx for new in leftv_y]
lines_x.append(leftv_x)
lines_y.append(leftv_y)

rightv_y = [yy for yy in range(starty,endy+1)]
rightv_x = [endx for new in rightv_y]
lines_x.append(rightv_x)
lines_y.append(rightv_y)

plt.plot(lines_x,lines_y,'go')

plt.show()

#Plotting Ground truth/Actual output
plt.imshow(Y_test[choice:choice+1,:,:,0].reshape(16,16),cmap = 'gray')
plt.title('Ground truth Output Y')
plt.show()

#Plotting predicted output
plt.imshow(yout[0,:,:,0].reshape(16,16),cmap = 'gray')
plt.title('Predicted Output Y: Heat Map')
plt.show()




#Actual max value in the ground truth labels (=1 since a cell containing center of nodule was assigned 1)
print('ACTUAL TRANSFORMED CO-ORDINATES OF NODULE POSITION: ',np.where(Y_test[choice:choice+1,:,:,0]==1))

#Plotting the highest few maximum values in the predicted output

top5 = []
for last in range(1,2):
    print('\nPREDICTED TOP '+str(last) +': ',np.where(yout[0,:,:,0]==sorted(yout[0,:,:,0].ravel())[-last]))
    top5.append(np.where(yout[0,:,:,0]==sorted(yout[0,:,:,0].ravel())[-last]))
    
#Plotting input image with top 5 predicted boundaries
plt.imshow(test_image_1.reshape(512,512),cmap = 'gray')
for y,x in top5:

    lines_x = []
    lines_y = []
    startx = x[0]*32
    endx = startx + 32
    starty = y[0]*32
    endy = starty + 32
#     test_image_1.reshape(512,512)= cv2.circle(test_image_1.reshape(512,512),(startx,starty),(endx,endy),(0,255,0),1)
    toph_x = [xx for xx in range(startx,endx+1)]
    toph_y = [starty for new in toph_x]
    lines_x.append(toph_x)
    lines_y.append(toph_y)
    
    bottomh_x = [xx for xx in range(startx,endx+1)]
    bottomh_y = [endy for new in bottomh_x]
    lines_x.append(bottomh_x)
    lines_y.append(bottomh_y)
    
    leftv_y = [yy for yy in range(starty,endy+1)]
    leftv_x=[startx for new in leftv_y]
    lines_x.append(leftv_x)
    lines_y.append(leftv_y)
    
    rightv_y = [yy for yy in range(starty,endy+1)]
    rightv_x = [endx for new in rightv_y]
    lines_x.append(rightv_x)
    lines_y.append(rightv_y)
    
    #Plotting bounding box
    plt.plot(lines_x,lines_y,'go')
    
    
plt.title('Input X with top 1 predicted nodules')
plt.show()

ValueError: Error when checking input: expected conv2d_1_input to have 4 dimensions, but got array with shape (1, 512, 512)